In [37]:
import pandas as pd
import os
from google.cloud import bigquery

In [41]:
#Nom du projet name et table_id à récupérer sur GCP
project="marbotic"
table_id = "users_metrics"

In [39]:
#intégration des credentials 
from google.oauth2 import service_account

key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [103]:
def get_table_names(project,credentials):
    '''
    Returning a list of all the table nanme of a bigquery database
    '''
    client = bigquery.Client(project=project, credentials=credentials)
    
    
    query = f"""SELECT table_name FROM {project}.marbotic_dataset.INFORMATION_SCHEMA.TABLES"""


    query_job = client.query(query)
    results=query_job.result()

    return [row[0] for row in results]

In [104]:
def extract_table(project,credentials,table):
    '''
    Query the selected table of the selected project on bigquery
    '''
    client = bigquery.Client(project=project, credentials=credentials)
    
    table_dest = f"`{project}.marbotic_dataset.{table}`"
    
    query = f"""
    SELECT *
    FROM {table_dest}
    """

    query_job = client.query(query)  # Make an API request.
    results=query_job.result().to_dataframe() # Waits for job to complete.

    return pd.DataFrame(results)

In [105]:
def bq_to_csv(project,credentials,table):
    '''
    Saving to csv all the table from a biqquery database
    '''
    outname = f'{table}.csv'

    outdir = '../data'
    if not os.path.exists(outdir):
        os.mkdir(outdir)

    fullname = os.path.join(outdir, outname)
    extract_table(project,credentials,table).to_csv(fullname,index=False)
    print(f"File {outname} created in {outdir}")

In [106]:
#Fetching all the table name of our project
table_list = get_table_names(project,credentials)

In [108]:
#Importing all table from bigquery and saving localy in csv file
for table in table_list:
    bq_to_csv(project,credentials,table)

File user_id.csv created in ../data
File users_metrics.csv created in ../data
File event_properties.csv created in ../data
File sessions_metrics.csv created in ../data
File session_augmented.csv created in ../data
File events.csv created in ../data
File user_properties.csv created in ../data


In [120]:
#To control if import was done correctly
pd.read_csv('../data/session_augmented.csv')

,session_id,city,language,os_version,region,country,device_model,start_version,session_end,id,...,Action_Element_Name,Profile_Age_days,good_answer_ratio,duration_min,user_creation_time,previous_session,time_since_last_session_min,week_maturity,churn_risk,Type
0,1.646929e+12,Bourg-en-Bresse,French,15.3.1,Ain,France,iPad 6,1.7.2,2022-03-10 16:22:43.368000+00:00,53,...,Home,1579.0,88.888889,5.122283,2021-11-12 15:53:03.441000+00:00,6,10094.564267,18,1,Parent
1,1.648985e+12,Toulon,English,12.5.5,Var,France,iPad Mini 2,1.3.0,2022-04-03 11:51:20.277000+00:00,101,...,Back,1721.0,77.777778,25.490850,2022-03-27 13:22:17.540000+00:00,3,4586.193183,2,1,Parent
2,1.647342e+12,Zug,English,15.1,Zug,Switzerland,"iPad11,6",4.2.2,2022-03-15 11:00:21.164000+00:00,16,...,Adults,-1.0,42.857143,1.294250,2021-11-09 10:31:37.463000+00:00,2,2.259583,19,0,Teacher
3,1.647341e+12,Zug,English,15.1,Zug,Switzerland,"iPad11,6",4.2.2,2022-03-15 11:00:46.666000+00:00,43,...,Back,1222.0,40.000000,10.480217,2021-11-09 10:31:37.463000+00:00,0,0.000000,19,0,Teacher
4,1.647342e+12,Zug,English,15.1,Zug,Switzerland,"iPad11,6",4.2.2,2022-03-15 10:59:04.869000+00:00,30,...,Back,-1.0,42.857143,2.282250,2021-11-09 10:31:37.463000+00:00,1,6.501350,19,0,Teacher
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18811,1.648207e+12,Central,English,14.7.1,Central and Western District,Hong Kong,"iPad8,12",1.3.0,2022-03-25 11:24:41.909000+00:00,1,...,NaN,NaN,NaN,0.000000,2021-03-31 13:12:05.143000+00:00,15,347.674633,53,0,Parent
18812,1.648256e+12,Central,English,14.7.1,Central and Western District,Hong Kong,"iPad8,12",1.3.0,2022-03-26 01:03:26.253000+00:00,1,...,NaN,NaN,NaN,0.000000,2021-03-31 13:12:05.143000+00:00,17,17.747017,53,0,Parent
18813,1.648283e+12,Central,English,14.7.1,Central and Western District,Hong Kong,"iPad8,12",1.3.0,2022-03-26 08:30:01.952000+00:00,1,...,NaN,NaN,NaN,0.000000,2021-03-31 13:12:05.143000+00:00,19,433.337083,53,0,Parent
18814,1.647066e+12,Central,Chinese,12.5.5,Central and Western District,Hong Kong,iPad Mini 2,4.1.0,2022-03-12 06:23:50.078000+00:00,1,...,NaN,NaN,NaN,0.000000,2020-12-15 08:34:55.985000+00:00,1,64.237017,66,0,Parent
